# SQLAlchemy Example
An example of object-relational mapping (ORM) using SQLAlchemy.

#### Import packages and modules used in subsequent cells.

In [1]:
import configparser
import sqlalchemy
import sqlalchemy_utils

#### Define a function for parsing database connection parameters from a configuration file.

In [2]:
def _database_config(config_file, section='postgresql'):

    # create a parser
    config_parser = configparser.ConfigParser()

    # read configuration file
    config_parser.read(config_file)

    # get section, default to postgresql
    db_config = {}
    if config_parser.has_section(section):
        params = config_parser.items(section)
        for param in params:
            db_config[param[0]] = param[1]
    else:
        raise Exception(f'Section {section} not found in the {config_file} file')

    return db_config

#### Read database connection parameters and use these to configure an SQLAlchemy Engine instance.

In [3]:
params = _database_config("C:/home/data/pullpoint/database.ini")
db_connection_details = f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['database']}"
engine = sqlalchemy.create_engine(db_connection_details, echo=True)

#### Create an instance of the declarative base class and define as associated mapped class for notifications.

In [4]:
Base = sqlalchemy.ext.declarative.declarative_base()

class Notification(Base):
    
    __tablename__ = 'notifications'
    
    # object attributes (columns in each row)
    notification_id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True, autoincrement=True)
    ip_address = sqlalchemy.Column(sqlalchemy_utils.IPAddressType)
    message = sqlalchemy.Column(sqlalchemy.Unicode(255))
    
    # simple representation
    def __repr__(self):
        return f"<Notification(notification_id={self.notification_id}, ip_address={self.ip_address}, message={self.message})>"